In [ ]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns

warnings.filterwarnings('ignore')
sns.set_theme(style='whitegrid', palette='viridis')
plt.rcParams.update({'figure.dpi': 120, 'axes.titlesize': 13, 'axes.labelsize': 11})

DATA_PATH = '../data/processed/listings_clean.csv'
df = pd.read_csv(DATA_PATH)

# ── Label mappings (from original categorical encoding) ─────────────────────
ROOM_LABELS = {
    0: 'Entire home/apt',
    1: 'Hotel room',
    2: 'Private room',
    3: 'Shared room',
}
NEIGHBOURHOOD_LABELS = {
    0:  'Batignolles-Monceau',
    1:  'Bourse',
    2:  'Buttes-Chaumont',
    3:  'Buttes-Montmartre',
    4:  'Entrepôt',
    5:  'Gobelins',
    6:  'Hôtel-de-Ville',
    7:  'Louvre',
    8:  'Luxembourg',
    9:  'Ménilmontant',
    10: 'Observatoire',
    11: 'Opéra',
    12: 'Palais-Bourbon',
    13: 'Panthéon',
    14: 'Passy',
    15: 'Popincourt',
    16: 'Reuilly',
    17: 'Temple',
    18: 'Vaugirard',
    19: 'Élysée',
}

df['room_type_label']   = df['room_type'].map(ROOM_LABELS)
df['neighbourhood_name'] = df['neighbourhood_cleansed'].map(NEIGHBOURHOOD_LABELS)

print('Dataset loaded:', df.shape)
df.head(3)

# Airbnb Paris — Exploratory Data Analysis
**Dataset:** `data/processed/listings_clean.csv`  
**Rows:** 55 655 listings · **Features:** 10 (after preprocessing)

---
## 1. Overview

In [ ]:
print('Shape:', df.shape)
print()
print('Dtypes:')
print(df.dtypes)
print()
print('Missing values:')
print(df.isnull().sum())

In [ ]:
df.describe().round(2)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 4))
mask = df.drop(columns=['room_type_label', 'neighbourhood_name']).isnull()
sns.heatmap(
    mask,
    cbar=False,
    yticklabels=False,
    cmap='viridis',
    ax=ax,
)
ax.set_title('Missing Values Heatmap (yellow = missing)')
plt.tight_layout()
plt.show()
print('→ No missing values after preprocessing.')

---
## 2. Distribution de la target : `price`

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ── Raw price ───────────────────────────────────────────────────────────────
sns.histplot(df['price'], kde=True, bins=100, color='steelblue', ax=axes[0])
axes[0].set_title('Distribution brute de price')
axes[0].set_xlabel('Price (€/nuit)')
axes[0].set_ylabel('Count')
axes[0].xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'€{x:,.0f}'))

# ── log(price) ──────────────────────────────────────────────────────────────
log_price = np.log1p(df['price'])
sns.histplot(log_price, kde=True, bins=80, color='darkorange', ax=axes[1])
axes[1].set_title('Distribution de log(price + 1)')
axes[1].set_xlabel('log(price + 1)')
axes[1].set_ylabel('Count')

plt.suptitle('Price : distribution brute vs log-transformée', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
order = df.groupby('room_type_label')['price'].median().sort_values(ascending=False).index
sns.boxplot(
    data=df[df['price'] <= df['price'].quantile(0.99)],
    x='room_type_label',
    y='price',
    order=order,
    palette='viridis',
    ax=ax,
)
ax.set_title('Distribution du prix par room_type (outliers > p99 exclus)')
ax.set_xlabel('Type de logement')
ax.set_ylabel('Price (€/nuit)')
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'€{x:,.0f}'))
plt.tight_layout()
plt.show()

**Conclusion :** La distribution brute est très *right-skewed* (quelques outliers à €30 000/nuit). Après `log1p`, la distribution se rapproche d'une gaussienne. **Recommandation : entraîner XGBoost sur `log(price)` puis inverser avec `expm1` pour les prédictions.**

---
## 3. Features catégorielles

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# ── Listings par room_type ───────────────────────────────────────────────────
counts = df['room_type_label'].value_counts()
bars = axes[0].bar(counts.index, counts.values, color=sns.color_palette('viridis', len(counts)))
axes[0].bar_label(bars, fmt='%d', padding=4)
axes[0].set_title('Nombre de listings par room_type')
axes[0].set_xlabel('Type de logement')
axes[0].set_ylabel('Nombre de listings')
axes[0].tick_params(axis='x', rotation=15)

# ── Médiane price par room_type ──────────────────────────────────────────────
med = df.groupby('room_type_label')['price'].median().sort_values(ascending=False)
bars2 = axes[1].bar(med.index, med.values, color=sns.color_palette('viridis', len(med)))
axes[1].bar_label(bars2, fmt='€%.0f', padding=4)
axes[1].set_title('Prix médian par room_type')
axes[1].set_xlabel('Type de logement')
axes[1].set_ylabel('Prix médian (€/nuit)')
axes[1].tick_params(axis='x', rotation=15)

plt.suptitle('Room type : volume et prix', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 7))

# ── Top 15 quartiers par prix médian ────────────────────────────────────────
med_by_neigh = (
    df.groupby('neighbourhood_name')['price']
    .median()
    .sort_values(ascending=False)
    .head(15)
)
colors = sns.color_palette('viridis', len(med_by_neigh))
bars = axes[0].barh(med_by_neigh.index[::-1], med_by_neigh.values[::-1], color=colors[::-1])
axes[0].bar_label(bars, fmt='€%.0f', padding=4)
axes[0].set_title('Top 15 quartiers — Prix médian')
axes[0].set_xlabel('Prix médian (€/nuit)')
axes[0].set_ylabel('')

# ── Top 15 quartiers par volume ──────────────────────────────────────────────
count_by_neigh = (
    df['neighbourhood_name']
    .value_counts()
    .head(15)
)
colors2 = sns.color_palette('viridis', len(count_by_neigh))
bars2 = axes[1].barh(count_by_neigh.index[::-1], count_by_neigh.values[::-1], color=colors2[::-1])
axes[1].bar_label(bars2, fmt='%d', padding=4)
axes[1].set_title('Top 15 quartiers — Nombre de listings')
axes[1].set_xlabel('Nombre de listings')
axes[1].set_ylabel('')

plt.suptitle('Analyse par quartier (arrondissement)', fontsize=14, y=1.01)
plt.tight_layout()
plt.show()

---
## 4. Features numériques vs price

In [ ]:
df_plot = df[df['price'] <= df['price'].quantile(0.99)].copy()

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

scatter_cfg = dict(alpha=0.15, s=8, color='steelblue')
line_cfg    = dict(color='crimson', linewidth=2)

for ax, col, xlabel in zip(
    axes,
    ['accommodates', 'bedrooms', 'availability_365'],
    ['Accommodates (personnes)', 'Bedrooms', 'Availability (jours/an)'],
):
    ax.scatter(df_plot[col], df_plot['price'], **scatter_cfg)
    m, b = np.polyfit(df_plot[col], df_plot['price'], 1)
    x_line = np.linspace(df_plot[col].min(), df_plot[col].max(), 100)
    ax.plot(x_line, m * x_line + b, **line_cfg, label=f'slope={m:.1f}')
    ax.set_title(f'{col} vs price')
    ax.set_xlabel(xlabel)
    ax.set_ylabel('Price (€/nuit)')
    ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'€{x:,.0f}'))
    ax.legend(fontsize=9)

plt.suptitle('Features numériques vs price (outliers > p99 exclus)', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))

df_bed = df_plot.copy()
df_bed['bedrooms_capped'] = df_bed['bedrooms'].clip(upper=5).astype(int)
df_bed['bedrooms_label'] = df_bed['bedrooms_capped'].apply(lambda x: f'{x}+' if x == 5 else str(x))

order = sorted(df_bed['bedrooms_label'].unique(), key=lambda x: int(x.replace('+', '')))
sns.boxplot(
    data=df_bed,
    x='bedrooms_label',
    y='price',
    order=order,
    palette='viridis',
    ax=ax,
)
ax.set_title('Distribution du prix par nombre de chambres (outliers > p99 exclus)')
ax.set_xlabel('Nombre de chambres')
ax.set_ylabel('Price (€/nuit)')
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'€{x:,.0f}'))
plt.tight_layout()
plt.show()

---
## 5. Corrélations

In [ ]:
numeric_cols = ['price', 'room_type', 'neighbourhood_cleansed', 'accommodates',
                'bedrooms', 'number_of_reviews', 'review_scores_rating',
                'availability_365', 'minimum_nights', 'bathrooms']

corr = df[numeric_cols].corr()

fig, ax = plt.subplots(figsize=(12, 9))
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(
    corr,
    mask=mask,
    annot=True,
    fmt='.2f',
    cmap='RdYlGn',
    center=0,
    square=True,
    linewidths=0.5,
    cbar_kws={'shrink': 0.8},
    ax=ax,
)
ax.set_title('Matrice de corrélation (triangle inférieur)', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

corr_with_price = (
    corr['price']
    .drop('price')
    .sort_values()
)
colors = ['crimson' if v < 0 else 'steelblue' for v in corr_with_price.values]
bars = ax.barh(corr_with_price.index, corr_with_price.values, color=colors)
ax.bar_label(bars, fmt='%.3f', padding=4)
ax.axvline(0, color='black', linewidth=0.8)
ax.set_title('Corrélation de chaque feature avec price')
ax.set_xlabel('Coefficient de corrélation de Pearson')
ax.set_xlim(-0.25, 0.65)
plt.tight_layout()
plt.show()

---
## 6. Outliers

In [ ]:
p99 = df['price'].quantile(0.99)
outliers = df[df['price'] > p99]

print(f'Seuil p99 : €{p99:,.0f}/nuit')
print(f'Outliers  : {len(outliers)} listings ({len(outliers)/len(df):.1%} du dataset)')
print()
print('Stats outliers :')
print(outliers['price'].describe().round(0))

In [ ]:
df_capped = df.copy()
df_capped['price_capped'] = df_capped['price'].clip(upper=p99)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

sns.histplot(df['price'], bins=120, kde=True, color='crimson', ax=axes[0])
axes[0].axvline(p99, color='black', linestyle='--', linewidth=1.5, label=f'p99 = €{p99:,.0f}')
axes[0].set_title('Avant capping (outliers inclus)')
axes[0].set_xlabel('Price (€/nuit)')
axes[0].legend()
axes[0].xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'€{x:,.0f}'))

sns.histplot(df_capped['price_capped'], bins=80, kde=True, color='steelblue', ax=axes[1])
axes[1].axvline(p99, color='black', linestyle='--', linewidth=1.5, label=f'Cap = €{p99:,.0f}')
axes[1].set_title(f'Après capping à p99 (€{p99:,.0f})')
axes[1].set_xlabel('Price (€/nuit)')
axes[1].legend()
axes[1].xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'€{x:,.0f}'))

plt.suptitle('Impact du capping des outliers sur la distribution du prix', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

print(f'\n→ Seuil recommandé : €{p99:,.0f}/nuit (p99). Retirer ou capper ces {len(outliers)} listings avant entraînement.')

---
## 7. Conclusions

### 5 insights clés pour le modèle XGBoost

**1. Log-transformer `price`**  
La target est fortement *right-skewed* (mean ≈ €252, max ≈ €30 400). Entraîner sur `log1p(price)` puis prédire avec `expm1()` améliorera la convergence et les métriques.

**2. `accommodates` et `bedrooms` sont les features les mieux corrélées au prix**  
Ces deux variables ont les corrélations les plus fortes avec `price`. Le feature engineering sur la capacité (ex. `price_per_person = price / accommodates`) peut aider.

**3. `room_type` est discriminant**  
*Entire home/apt* a un prix médian 2× supérieur à *Private room*. C'est une feature catégorielle à fort impact malgré son encoding ordinal.

**4. `neighbourhood_cleansed` apporte de la variance géographique**  
Les quartiers du centre (Louvre, Élysée, Palais-Bourbon) ont des prix médians significativement plus élevés. L'encodage ordinal actuel est acceptable pour XGBoost (insensible à l'ordre), mais un target encoding pourrait améliorer les performances.

**5. Capper les outliers à p99 (€1 700/nuit)**  
557 listings (1%) tirent la distribution vers la droite. Les exclure ou les capper avant entraînement réduit le risque d'overfitting sur des cas atypiques (villas de luxe, événements spéciaux).

### Recommandations de preprocessing supplémentaire

| Action | Justification |
|---|---|
| `y = np.log1p(price)` | Distribution plus gaussienne, meilleures métriques |
| Cap `price` à €1 700 (p99) | Limiter l'impact des outliers extrêmes |
| Target-encoding de `neighbourhood_cleansed` | Capture mieux la relation prix/quartier qu'un label-encoding ordinal |
| Feature `price_per_person = price / accommodates` | Normalise la capacité |
| Cap `minimum_nights` à 30 | Les séjours longs (> 30 nuits) sont des cas atypiques |
| `review_scores_rating` → imputer par room_type median | Plus précis que la médiane globale |